## Imports

In [1]:
from google.cloud import pubsub
from google.api_core import exceptions as gexc

## Parameters

In [2]:
#@title Google Cloud Project Info { display-mode: "form" }
project_id = "strokach-playground" #@param {type:"string"}

In [3]:
try:
    from google.colab import auth
    auth.authenticate_user()
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

## Workflow

In [4]:
sub_client = pubsub.SubscriberClient()

In [5]:
subscription_name = "projects/{}/subscriptions/taxirides-realtime-sub".format(project_id)

try:
    sub_client.create_subscription(
        subscription_name,
        "projects/pubsub-public-data/topics/taxirides-realtime",
    )
except gexc.AlreadyExists:
    sub_client.delete_subscription(subscription_name)
    sub_client.create_subscription(
        subscription_name,
        "projects/pubsub-public-data/topics/taxirides-realtime",
    )

In [6]:
requests = [{"subscription": subscription_name, "stream_ack_deadline_seconds": 10}]

In [7]:
for response in sub_client.streaming_pull(requests=requests):
    print(response)

Unknown: None Exception iterating requests!